# Calculate Taylor Factor (TF) and Schmid Factor (SF) of orientation(s) wrt. axisymmetric tensile deformation

Uses grain refOri to calculate TF and SF

## Import modules

In [25]:
import numpy as np
import matplotlib.pyplot as plt

from skimage import transform as tf
from skimage import io

import hrdic
import ebsd
import quat

import copy

%matplotlib osx

## Load DIC data

In [2]:
#Load in DIC map
DicFilePath = "./example_data_AH/"
DicMap = hrdic.Map(DicFilePath, "hrdicData.txt")

Loaded DaVis 8.1.5 data (dimensions: 1860 x 1628 pixels, sub-window size: 8 x 8 pixels)


## Load EBSD data

In [3]:
mapFiles = ["./example_data_AH/ebsdData"]

maps = []

for i, mapFile in enumerate(mapFiles):
    maps.append(ebsd.Map(mapFile, "cubic"))
    maps[i].loadData(mapFile, "cubic")
    maps[i].binData = maps[i].binData[::-1]  #rotate the map 180 degrees
    maps[i].buildQuatArray()
    maps[i].findBoundaries(boundDef = 6)
    maps[i].findGrains(minGrainSize=10)
    maps[i].calcGrainMisOri(calcAxis = True)

Loaded EBSD data (dimensions: 612 x 465 pixels, step size: 0.5 um)
Error with cpr file. Number of phases mismatch.
Loaded EBSD data (dimensions: 612 x 465 pixels, step size: 0.5 um)


## Link DIC and EBSD data

In [4]:
# load the disMap crop parameters
DicMap.cropDists = np.loadtxt("./example_data_AH/dicMapCropDists.txt")

# set the dic map crop parameters
DicMap.cropDists_temp=[]
for p,q in DicMap.cropDists:
    p,q = int(p),int(q)
    DicMap.cropDists_temp.append([p,q])
DicMap.cropDists_temp=np.array(DicMap.cropDists_temp)
DicMap.cropDists=copy.deepcopy(DicMap.cropDists_temp)

# define the crop
DicMap.setCrop(xMin=DicMap.cropDists[0,0], xMax=DicMap.cropDists[0,1], yMin=DicMap.cropDists[1,0], yMax=DicMap.cropDists[1,1])

# load and define the homogPoints
DicMap.homogPoints = np.loadtxt("./example_data_AH/dicMapHomogPoints.txt")
maps[0].homogPoints = np.loadtxt("./example_data_AH/ebsdMapHomogPoints.txt")

# link the dic and ebsd data
DicMap.linkEbsdMap(maps[0])

# find the grains
DicMap.findGrains(minGrainSize=1434)

### Test the link has worked

In [9]:
DicMap.locateGrainID()

Grain ID: 13


In [10]:
# test the link has worked
grainNum = DicMap.currGrainId
DicMap.grainList[grainNum].plotMaxShear()
maps[0].grainList[DicMap.ebsdGrainIds[grainNum]].plotMisOri()

## Find the TF

In [45]:
def findPrincipleStresses(quats,eps):
    
    a=quats
    
    Q = quat.Quat.rotMatrix(a)
    Q_T = np.transpose(Q)
    
    eps=np.array(eps)
    Q_dot_eps=np.dot(Q,eps)
    Q_dot_eps_dot_Q_T=np.dot(Q_dot_eps,Q_T)
    
    principleStresses=Q_dot_eps_dot_Q_T
    
    return principleStresses

ebsd.Grain.findPrincipleStresses = findPrincipleStresses

In [42]:
def calcTF(self,strain=1,epsMapStrain=False,grainEps=None,customEPS=None):
    
    quats=self.refOri
    
    if epsMapStrain:
        
        xMin=DicMap.cropDists[0,0]
        xMax=DicMap.cropDists[0,1]
        yMin=DicMap.cropDists[1,0]
        yMax=DicMap.cropDists[1,1]

        meanU11=np.mean(DicMap.f11[yMin:-yMax,xMin:-xMax])
        meanU22=np.mean(DicMap.f22[yMin:-yMax,xMin:-xMax])

        normU11=meanU11/meanU11
        normU22=meanU22/meanU11
        normU33=-1*(1+normU22)
        
        eps=np.array([[normU11,0,0],[0,normU22,0],[0,0,normU33]])
        n = strain # tensile strain in x direction
        eps = [[j*n for j in i] for i in eps]
        
    elif grainEps is not None:
        eps=grainEps
        n = strain # tensile strain in x direction
        eps = [[j*n for j in i] for i in eps]   
        
    elif customEPS is not None:
        eps=customEPS
        n = strain
        eps = [[j*n for j in i] for i in eps] 
    
    else:
        eps=np.array([[1,0,0],[0,-0.5,0],[0,0,-0.5]])
        n = strain # tensile strain in x direction
        eps = [[j*n for j in i] for i in eps]

    # Burgers vector
    sS_b_symmeterise = np.array(
        [
            [0,1,-1],
            [-1,0,1],  
            [1,-1,0],  
            [0,-1,1], 
            [1,0,-1], 
            [-1,1,0],

            [1,-1,0],  
            [1,0,1],  
            [0,1,1],   
            [-1,0,-1],   
            [0,-1,-1],  
            [-1,1,0],  

            [0,1,-1], 
            [1,0,1], 
            [1,1,0], 
            [-1,0,-1], 
            [-1,-1,0], 
            [0,-1,1], 

            [-1,0,1],
            [1,1,0],   
            [0,1,1],   
            [-1,-1,0],  
            [0,-1,-1],  
            [1,0,-1]
        ]
    )

    # slip plane
    sS_n_symmeterise = np.array(
        [
            [1,1,1],
            [1,1,1],
            [1,1,1],
            [1,1,1],
            [1,1,1],
            [1,1,1],

            [1,1,-1],
            [1,1,-1],
            [1,1,-1],
            [1,1,-1],
            [1,1,-1],
            [1,1,-1],

            [-1,1,1],
            [-1,1,1],
            [-1,1,1],
            [-1,1,1],
            [-1,1,1],
            [-1,1,1],

            [1,-1,1],
            [1,-1,1],
            [1,-1,1],
            [1,-1,1],
            [1,-1,1],
            [1,-1,1]
        ]
    )


    eps = findPrincipleStresses(quats,eps)
    
    # calculate the deformation tensor for each sS
    sSeps_list=[]

    for b,n in zip(sS_b_symmeterise,sS_n_symmeterise):

        sS_b_norm = np.sqrt((b[0])**2 + (b[1])**2 + (b[2])**2)
        sS_b_normalise = b/sS_b_norm

        sS_n_norm = np.sqrt((n[0])**2 + (n[1])**2 + (n[2])**2)
        sS_n_normalise = n/sS_n_norm

        sSeps=np.tensordot(sS_b_normalise,sS_n_normalise,0)

        sSeps_list.append(sSeps)
    
    # initialise b
    b = np.zeros((1,len(sSeps_list)))

    # define CRSS for each sS
    CRSS = np.ones((len(sSeps_list),1))
    
    # reshape the deformation tensor
    A = []
    for sSep in sSeps_list:
        val_list = (0.5*(sSep+np.transpose(sSep)))
        val_list = val_list.flatten().tolist()
        A.append(val_list)
    
    # select 5 independent strain components from deformation tensor
    A_select=[]
    for i in A:
        As = i[0:3] + i[4:6]
        A_select.append(As)
    A = A_select
    A_T = np.transpose(A)
    
    # reshape strain coefficents
    y = eps.flatten().tolist()

    # select same 5 coefficients to match the independent 5 strain components
    y = y[0:3] + y[4:6]
    
    # relate the sS deformation tensor components to the orientation strain components with coefficient b
    # i.e., solve for b in A*b=y such that CRSS*b is minimal
    from scipy import optimize as opt
    for i in range(len(y)):
        b = opt.linprog(CRSS.flatten(), 
            A_eq=A_T, 
            b_eq=y, 
            bounds=[],
            method='simplex', 
            #callback=None, 
            #options={'disp': False, 'bland': False, 'tol': 1e-12, 'maxiter': 1000}
                       )
    # Taylor factor is then the sum of the coeffiencents b
    taylorFactor=b["fun"]
    
    return taylorFactor

ebsd.Grain.calcTF=calcTF

In [47]:
# test for one grain
maps[0].grainList[DicMap.ebsdGrainIds[DicMap.currGrainId]].calcTF()

3.1510538438050641

In [49]:
# calc TF for each grain
TFs=[]
for grain in maps[0].grainList:
    TF = grain.calcTF()
    TFs.append(TF)

In [50]:
# plot histogram of values
TF_hist=plt.hist(TFs,bins=20)
plt.xlabel("TF [  ]")
plt.ylabel("Number of grains")

## SFs

In [35]:
# calc SFs
maps[0].loadSlipSystems("./slip_systems/cubic.txt")
maps[0].calcAverageGrainSchmidFactors(loadVector=np.array([1, 0, 0]))

In [36]:
# record max SF for each grain
SFs=[]
for grain in maps[0].grainList:
    SF=np.max(grain.averageSchmidFactors)
    SFs.append(SF)

In [37]:
# plot histogram of values
SF_hist=plt.hist(SFs,bins=20)
plt.xlabel("SF [  ]")
plt.ylabel("Number of grains")